<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-harshrstgi24-cloud/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Harsh Rastogi

**Student Number**: 400513435

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    classification_report,
    accuracy_score
)
import plotly.express as px

# 1. Load data  (adjust filename/path if your CSV is named differently)
df = pd.read_excel("/content/W33836-XLS-ENG.xlsx") # Changed to pd.read_excel

# 2. Drop rows with missing values
df = df.dropna()

# 3. Drop constant columns (same value for all rows)
constant_cols = df.columns[df.nunique() == 1]
df = df.drop(columns=constant_cols)

# 4. Convert Dt_Customer to "days with company"
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"])
reference_date = df["Dt_Customer"].max()
df["Customer_Tenure_Days"] = (reference_date - df["Dt_Customer"]).dt.days

# Drop original date column (we keep the numeric tenure)
df = df.drop(columns=["Dt_Customer"])

df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Customer_Tenure_Days
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,4,7,0,0,0,0,0,0,1,971
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,2,5,0,0,0,0,0,0,0,125
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,10,4,0,0,0,0,0,0,0,472
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,4,6,0,0,0,0,0,0,0,65
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,6,5,0,0,0,0,0,0,0,321


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [3]:
# Target variable: spending on wines
target_col = "MntWines"

X = df.drop(columns=[target_col])
y = df[target_col]

# Identify numeric and categorical features
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_features = X.select_dtypes(exclude=["object", "category"]).columns.tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=20
)

len(X_train), len(X_test)


(1551, 665)

(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [5]:
# 1. Identify categorical and numeric feature columns from X
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_features = X.select_dtypes(exclude=["object", "category"]).columns.tolist()

# 2. Preprocessing: one-hot encode categoricals, pass numeric columns through
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features),
    ]
)

# 3. Random forest regressor with specified hyperparameters
rf_regressor = RandomForestRegressor(
    n_estimators=200,       # number of trees
    max_depth=10,          # maximum depth of any tree
    min_samples_split=4,   # min samples required to split a node
    min_samples_leaf=2,    # min samples in any leaf node
    random_state=20,
    n_jobs=-1
)

# 4. Pipeline: preprocessing + model
rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", rf_regressor),
    ]
)

# 5. Fit model to the training data
rf_pipeline.fit(X_train, y_train)

rf_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status']),
                                                 ('num', 'passthrough',
                                                  ['ID', 'Year_Birth', 'Income',
                                                   'Kidhome', 'Teenhome',
                                                   'Recency', 'MntFruits',
                                                   'MntMeatProducts',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth',
                                                   'AcceptedCmp3',
                                                   'AcceptedCmp4',
                                                   'AcceptedCmp5',
                                                   'AcceptedCmp1',
                                                   'AcceptedCmp2', 'Complain',
                                                   'Response',
                                                   'Customer_Tenure_Days'])])),
                ('model',
                 RandomForestRegressor(max_depth=10, min_samples_leaf=2,
                                       min_samples_split=4, n_estimators=200,
                                       n_jobs=-1, random_state=20))])

(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [18]:
# Start from an existing row so all required columns are present
new_customer = X_train.iloc[[0]].copy()

# Overwrite the features given in the table
new_customer.loc[:, "Age"] = 48
new_customer.loc[:, "Education"] = "Graduation"
new_customer.loc[:, "Marital_Status"] = "Married"
new_customer.loc[:, "Income"] = 80000
new_customer.loc[:, "Kidhome"] = 1
new_customer.loc[:, "Teenhome"] = 1

# Dt_Customer as given (string is fine – it will be treated as categorical)
if "Dt_Customer" in new_customer.columns:
    new_customer.loc[:, "Dt_Customer"] = "2016-10-10"

new_customer.loc[:, "Recency"] = 43
new_customer.loc[:, "NumDealsPurchases"] = 2
new_customer.loc[:, "NumWebPurchases"] = 1
new_customer.loc[:, "NumCatalogPurchases"] = 0
new_customer.loc[:, "NumStorePurchases"] = 15
new_customer.loc[:, "NumWebVisitsMonth"] = 5

# Set all campaign acceptance flags to 0 if they exist
for col in ["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3",
            "AcceptedCmp4", "AcceptedCmp5"]:
    if col in new_customer.columns:
        new_customer.loc[:, col] = 0

# Complain = 0 if the column exists
if "Complain" in new_customer.columns:
    new_customer.loc[:, "Complain"] = 0

# Predict spending on wines
predicted_wine_spend = rf_pipeline.predict(new_customer)[0]
print(f"Predicted spending on wines for this customer: {predicted_wine_spend:.2f}")

# Evaluate model on test set

y_pred_test = rf_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print("\nRandom Forest performance on test set:")
print(f"MSE  : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"MAE  : {mae:.2f}")
print(f"R^2  : {r2:.3f}")

Predicted spending on wines for this customer: 475.27

Random Forest performance on test set:
MSE  : 22735.13
RMSE : 150.78
MAE  : 86.36
R^2  : 0.776


To evaluate the model’s performance on the test dataset, I considered two commonly used regression metrics: Root Mean Squared Error (RMSE) and R-squared (R²).

RMSE = 150.78
RMSE measures the average magnitude of prediction error in the same units as the target variable. An RMSE of about 151 indicates that typical prediction errors are around ±$150. Since wine spending in this dataset has substantial variation (ranging from very low to very high amounts), this level of error is reasonable but shows that the model does not capture all the complexity behind consumer wine purchases.

R² = 0.776
R² shows how much of the variance in wine spending the model explains. A value of 0.776 means the model accounts for 77.6% of the variation in the target variable, which is relatively strong for consumer-behavior data. It suggests that the model is capturing meaningful relationships between demographic/purchasing features and wine spending.

Overall Assessment:
The model performs well in explaining a large proportion of the variability in wine spending (high R²), although prediction errors remain moderate (RMSE ~150), which is expected given the natural noise in customer purchasing behavior. The Random Forest model therefore provides a solid balance of accuracy and interpretability for this task.

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [7]:
# Using negative MSE because sklearn requires a "higher is better" score;
# we will convert it to RMSE afterward.
neg_mse_scores = cross_val_score(
    rf_pipeline,
    X,
    y,
    scoring="neg_mean_squared_error",
    cv=6,
    n_jobs=-1
)

# Convert negative MSE → positive RMSE
rmse_scores = np.sqrt(-neg_mse_scores)

print("RMSE scores for each fold:")
print(rmse_scores)

print("\nMean RMSE across 6 folds:", rmse_scores.mean())
print("Standard deviation of RMSE :", rmse_scores.std())

RMSE scores for each fold:
[132.42352714 147.93306239 165.4005864  141.99850013 151.79384109
 141.48184751]

Mean RMSE across 6 folds: 146.83856077557945
Standard deviation of RMSE : 10.248701470962246


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [8]:
# Hyperparameter grid (note the "model__" prefix because the RF is inside the pipeline)
param_grid = {
    "model__n_estimators": [50, 100],      # number of trees
    "model__max_depth": [5, 10, 15],       # max depth
    "model__min_samples_split": [3, 6],    # min samples to split a node
    "model__min_samples_leaf": [2, 4, 8],  # min samples in a leaf
}

# Grid search with 5-fold CV, using negative MSE as the score
grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Best hyperparameter combination
print("Best hyperparameters found:")
print(grid_search.best_params_)

# Corresponding performance score (convert neg MSE -> RMSE)
best_neg_mse = grid_search.best_score_
best_rmse = np.sqrt(-best_neg_mse)

print("\nBest mean CV score (negative MSE):", best_neg_mse)
print("Best mean CV RMSE:", best_rmse)

# Retrieve the best model (pipeline with tuned RF)
best_rf_model = grid_search.best_estimator_
best_rf_model

Best hyperparameters found:
{'model__max_depth': 15, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 100}

Best mean CV score (negative MSE): -24700.67244977023
Best mean CV RMSE: 157.16447578816985


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status']),
                                                 ('num', 'passthrough',
                                                  ['ID', 'Year_Birth', 'Income',
                                                   'Kidhome', 'Teenhome',
                                                   'Recency', 'MntFruits',
                                                   'MntMeatProducts',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth',
                                                   'AcceptedCmp3',
                                                   'AcceptedCmp4',
                                                   'AcceptedCmp5',
                                                   'AcceptedCmp1',
                                                   'AcceptedCmp2', 'Complain',
                                                   'Response',
                                                   'Customer_Tenure_Days'])])),
                ('model',
                 RandomForestRegressor(max_depth=15, min_samples_leaf=2,
                                       min_samples_split=3, n_jobs=-1,
                                       random_state=20))])

To improve the Random Forest model, I performed hyperparameter tuning using GridSearchCV with a 5-fold cross-validation procedure. The grid searched over the following values:

Number of trees (n_estimators): 50, 100

Maximum tree depth (max_depth): 5, 10, 15

Minimum samples required to split a node (min_samples_split): 3, 6

Minimum samples in a leaf node (min_samples_leaf): 2, 4, 8

The performance metric used was negative mean squared error (neg_mean_squared_error), which scikit-learn maximizes. I then converted the best score back to RMSE for easier interpretation.

The grid search selected the following hyperparameter combination as the best:

n_estimators = 100

max_depth = 15

min_samples_split = 3

min_samples_leaf = 2

This configuration achieved a best mean cross-validated score of −24,780.67, which corresponds to a mean RMSE of approximately 157.16 across the folds. The corresponding pipeline (stored as best_rf_model) is the tuned Random Forest model that I will use as the “best” model for subsequent comparisons.

# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [9]:
# 1. Linear regression model using all relevant features
lr_model = LinearRegression()

# Re-use the same preprocess object (OneHotEncoder + passthrough numeric)
lr_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", lr_model),
    ]
)

# 2. Fit linear regression to the training data
lr_pipeline.fit(X_train, y_train)

# 3. Evaluate performance on the test data
y_pred_lr = lr_pipeline.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression performance on test set:")
print(f"  RMSE : {rmse_lr:.2f}")
print(f"  MAE  : {mae_lr:.2f}")
print(f"  R^2  : {r2_lr:.3f}")

# 4. For comparison: evaluate best RF model from Question 1 on the same test set
y_pred_best_rf_test = best_rf_model.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_best_rf_test)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_best_rf_test)
r2_rf = r2_score(y_test, y_pred_best_rf_test)

print("\nBest Random Forest (from GridSearchCV) on test set:")
print(f"  RMSE : {rmse_rf:.2f}")
print(f"  MAE  : {mae_rf:.2f}")
print(f"  R^2  : {r2_rf:.3f}")

Linear Regression performance on test set:
  RMSE : 177.81
  MAE  : 117.93
  R^2  : 0.689

Best Random Forest (from GridSearchCV) on test set:
  RMSE : 151.02
  MAE  : 86.00
  R^2  : 0.776


After fitting a linear regression model to the training data, I evaluated its performance on the same test set used in Question 1. The model achieved an RMSE of 177.81, MAE of 117.93, and an R² of 0.689. These results indicate that although the linear model captures some of the variation in wine spending, its prediction errors are noticeably larger than those of the tuned Random Forest model.

In comparison, the best Random Forest model found in Question 1 (via GridSearchCV) achieved a lower RMSE of 151.02 and a lower MAE of 86.00, along with a higher R² value of 0.776. These improvements across both error metrics suggest that the Random Forest model provides more accurate and more reliable predictions. Its ability to model non-linear relationships and interactions among features allows it to better fit the underlying purchasing behaviors in the dataset.

Overall, based on the two performance measures considered in Question 1 (RMSE and R²), the Random Forest model clearly outperforms linear regression, offering both smaller prediction errors and stronger explanatory power.

(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [17]:
# compute absolute residuals on the test set
abs_resid_lr = np.abs(y_pred_lr - y_test.values)
abs_resid_rf = np.abs(y_pred_best_rf_test - y_test.values)

# Put in a DataFrame for plotting
resid_df = pd.DataFrame({
    "abs_resid_lr": abs_resid_lr,
    "abs_resid_rf": abs_resid_rf,
})

# scatterplot (LR residuals vs RF residuals)
fig = px.scatter(
    resid_df,
    x="abs_resid_lr",
    y="abs_resid_rf",
    labels={
        "abs_resid_lr": "Absolute residuals (Linear Regression)",
        "abs_resid_rf": "Absolute residuals (Random Forest)",
    },
    title="Q–Q style comparison of absolute residuals: LR vs Random Forest"
)

# add 45-degree reference line
min_val = 0
max_val = max(resid_df["abs_resid_lr"].max(), resid_df["abs_resid_rf"].max()) * 1.05

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(width=2, dash="dash")
)

fig.update_layout(
    xaxis_range=[min_val, max_val],
    yaxis_range=[min_val, max_val],
)

fig.show()

This scatterplot compares the absolute residuals from the linear regression model (x-axis) with those from the tuned Random Forest model (y-axis) for the same test observations. The dashed 45-degree line represents the points where both models make errors of the same magnitude.

Points that lie below the line correspond to cases where the Random Forest has smaller errors than linear regression; points above the line indicate observations where linear regression performs better. In my plot, most points are concentrated below or close to the 45-degree line, especially for larger residual values, which suggests that the Random Forest tends to produce smaller prediction errors more often than the linear model. This visual evidence is consistent with the earlier numerical comparison (lower RMSE and higher R² for Random Forest), confirming that the Random Forest model provides more accurate predictions overall.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [16]:
# Compute 33.33% and 66.67% percentiles for meat spending
q_low, q_high = df["MntMeatProducts"].quantile([1/3, 2/3])

print("33.33% percentile:", q_low)
print("66.67% percentile:", q_high)

# Define a labeling function
def label_meat_spending(value):
    if value < q_low:
        return "low"
    elif value > q_high:
        return "high"
    else:
        return "medium"

# Apply labels to create a new column
df["Meat_Spend_Class"] = df["MntMeatProducts"].apply(label_meat_spending)

df["Meat_Spend_Class"].value_counts()

33.33% percentile: 24.0
66.67% percentile: 146.33333333333258


,count
Meat_Spend_Class,
medium,746
high,739
low,731


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [12]:
# Features: everything except the original meat spending and the class label
X_cls = df.drop(columns=["MntMeatProducts", "Meat_Spend_Class"])
y_cls = df["Meat_Spend_Class"]

# Train–test split (same style as before)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls,
    y_cls,
    test_size=0.3,
    random_state=20,
    stratify=y_cls   # keep class proportions similar in train and test
)

# Identify numeric and categorical columns
numeric_features_cls = X_cls.select_dtypes(exclude=["object", "category"]).columns.tolist()
categorical_features_cls = X_cls.select_dtypes(include=["object", "category"]).columns.tolist()

# Preprocessing: scale numeric features, one-hot encode categoricals
numeric_transformer_cls = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_transformer_cls = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor_cls = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_cls, numeric_features_cls),
        ("cat", categorical_transformer_cls, categorical_features_cls),
    ]
)

# K-nearest-neighbors classifier (k=5 as a reasonable default)
knn_clf = KNeighborsClassifier(n_neighbors=5)

# Full pipeline
knn_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor_cls),
        ("model", knn_clf),
    ]
)

# Fit the model
knn_pipeline.fit(X_train_cls, y_train_cls)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['ID', 'Year_Birth', 'Income',
                                                   'Kidhome', 'Teenhome',
                                                   'Recency', 'MntWines',
                                                   'MntFruits',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth',
                                                   'AcceptedCmp3',
                                                   'AcceptedCmp4',
                                                   'AcceptedCmp5',
                                                   'AcceptedCmp1',
                                                   'AcceptedCmp2', 'Complain',
                                                   'Response',
                                                   'Customer_Tenure_Days']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model', KNeighborsClassifier())])

(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [15]:
# Predict on the test set
y_pred_cls = knn_pipeline.predict(X_test_cls)

# Overall accuracy
acc = accuracy_score(y_test_cls, y_pred_cls)
print(f"Accuracy on test set: {acc:.3f}\n")

# Detailed classification report
print("Classification report for Meat_Spend_Class (low / medium / high):")
print(classification_report(y_test_cls, y_pred_cls))

Accuracy on test set: 0.770

Classification report for Meat_Spend_Class (low / medium / high):
              precision    recall  f1-score   support

        high       0.85      0.82      0.83       222
         low       0.79      0.87      0.83       219
      medium       0.67      0.62      0.65       224

    accuracy                           0.77       665
   macro avg       0.77      0.77      0.77       665
weighted avg       0.77      0.77      0.77       665



I built a K-nearest-neighbors (KNN) classification model within a preprocessing pipeline that standardizes numeric features and one-hot encodes categorical features. This ensures that distance calculations in KNN are meaningful. After fitting the model on the training data, I evaluated it on the test set.

The model achieved an overall accuracy of 0.770, meaning it correctly classified about 77% of customers into low, medium, or high meat-spending groups. The classification report shows that the model performs best on the low and high spending classes, both with F1-scores around 0.83. These groups have clearer spending patterns, making them easier to separate.

The medium class has a lower F1-score of 0.65, which is expected because medium spenders fall between the two percentile thresholds and share characteristics with both low and high groups. As a result, the model finds it harder to distinguish them cleanly.

Overall, the KNN model provides solid performance, especially for the low and high categories, while the medium class remains more challenging due to overlapping spending behavior.

# Describe how you used Gen. AI. in this assignment (2 points)

For this assignment, I used generative AI (ChatGPT) as a coding and learning assistant. I asked it to help me structure my Python code, understand how to apply machine learning techniques such as pipelines, cross-validation, GridSearchCV, and KNN classification, and clarify how to correctly interpret evaluation metrics. The AI provided code templates and explanations, which I reviewed, adapted, and tested myself in the notebook. All final decisions, model evaluations, interpretations, and written explanations were done by me based on my understanding of the course material.



**Note**: The remaining 5 points will be assigned to readability of the work.